<a href="https://colab.research.google.com/github/fuushyn/DL2CR/blob/main/683_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# data_path = "trace.csv"

# df = pd.read_csv(data_path)


ips_test=["A","B","B","C","D","E","A","F","D","E","F","C"]
# [1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 0, 0]

cache_size = 2   #2MB cache

import numpy as np
from tqdm import tqdm

def opt_gen(ips, cache_size):
    occ_vec = np.zeros(len(ips), dtype=int)
    last_used = {}
    with tqdm(total=len(ips), desc="Processing IPs") as pbar:
        for i, ip in enumerate(ips):
            pbar.update(1)
            if ip in last_used:
                last_use = last_used[ip]
                max_occ = np.max(occ_vec[last_use:i])
                if max_occ >= cache_size:
                    continue
                occ_vec[last_use:i] += 1
            last_used[ip] = i
    return occ_vec


def get_labels(occ_vec):
    occ_vec = np.array(occ_vec)
    diff = np.diff(occ_vec)
    labels = np.where(diff > 0, 1, 0)
    return np.concatenate(([0], labels))


In [16]:
ov = opt_gen(ips_test, cache_size)
print(ov)
print(get_labels(ov))

Processing IPs: 100%|██████████| 12/12 [00:00<00:00, 17427.86it/s]

[1 2 1 1 2 2 1 2 1 1 0 0]
[0 1 0 0 1 0 0 1 0 0 0 0]


In [17]:
trace_path = '/content/drive/MyDrive/CS683_project/sssp-3.csv'
df = pd.read_csv(trace_path, dtype={'Instruction Pointer': str,'Source Memory 0': str, 'Destination Memory 0': str, }, header=0, skiprows=[1])
print(len(df["Instruction Pointer"]))
df = df[(df['Source Memory 0'] != '0') | (df['Destination Memory 0'] != '0')]
print(len(df["Instruction Pointer"]))

ips = df["Instruction Pointer"]
ov = opt_gen(ips, cache_size=2000)
labels = get_labels(ov)
df["Labels"] = labels
df.head()

4769373
1526590


Processing IPs: 100%|██████████| 1526590/1526590 [00:14<00:00, 109017.50it/s]


,Instruction Pointer,Is Branch,Branch Taken,Destination Register 0,Destination Register 1,Source Register 0,Source Register 1,Source Register 2,Source Register 3,Destination Memory 0,Destination Memory 1,Source Memory 0,Source Memory 1,Source Memory 2,Source Memory 3,Labels
0,559889caf3b0,1,1,26,0.0,26.0,0.0,0.0,0.0,0,0.0,559889ebedc0,0.0,0.0,0.0,0
1,14e493afd1d0,0,0,6,0.0,15.0,6.0,0.0,0.0,7ffeb7771480,0.0,0,0.0,0.0,0.0,0
2,14e493afd1d2,0,0,6,0.0,5.0,6.0,0.0,0.0,7ffeb7771478,0.0,0,0.0,0.0,0.0,0
4,14e493afd1d6,0,0,6,0.0,7.0,6.0,0.0,0.0,7ffeb7771470,0.0,0,0.0,0.0,0.0,0
8,14e493afd1e3,0,0,0,0.0,6.0,8.0,0.0,0.0,7ffeb77713d0,0.0,0,0.0,0.0,0.0,0


In [18]:
(df['Labels']>0).sum()

666

In [19]:
np.max(ov)

531

In [20]:
#3print(df["Instruction Pointer"].nunique())

In [21]:
columns_to_keep = ['Instruction Pointer', 'Labels']
df = df[columns_to_keep]
df.head()


,Instruction Pointer,Labels
0,559889caf3b0,0
1,14e493afd1d0,0
2,14e493afd1d2,0
4,14e493afd1d6,0
8,14e493afd1e3,0


In [22]:
df.to_csv("cleaned_data_ssp-3.csv")

In [23]:
device = 'cuda'

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder

# ... (rest of your imports, if any)

# Assuming 'df' is your DataFrame and it has 'sequence' and 'label' columns
# where 'sequence' contains the string data and 'label' contains the corresponding labels.

# Convert string data to numerical indices using LabelEncoder
label_encoder = LabelEncoder()
df['sequence_encoded'] = label_encoder.fit_transform(df['Instruction Pointer'].astype(str))

# Define the sequence length (context_size)

# Prepare the dataset
X = []
y = []
context_size = 30  # The number of previous items to consider

# Number of sequences to use for testing
test_size = int(0.3 * len(df))  # For example, 20% for testing

# Define the cut-off point for the training and test set
train_cutoff = len(df) - test_size

# Create sequences of context_size previous items
for i in range(len(df) - context_size):
    X.append(df['sequence_encoded'].values[i:i+context_size])
    y.append(df['Labels'].values[i+context_size])

X = np.array(X)
y = np.array(y)

X_train, X_test = X[:train_cutoff], X[train_cutoff:]
y_train, y_test = y[:train_cutoff], y[train_cutoff:]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
batch_size = 64  # Adjust batch size to your needs

# Create TensorDatasets for training and testing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders for training and testing
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


# # Create TensorDatasets
# dataset = TensorDataset(X_tensor, y_tensor)

# # Create DataLoaders
# data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define model parameters
num_classes = 2  # Number of classes in your output
num_unique_strings = len(label_encoder.classes_)  # Number of unique strings
embedding_dim = 50  # Embedding dimensionality
hidden_size = 128  # LSTM hidden layer size
num_layers = 2  # Number of LSTM layers


# Define LSTM model with embedding layer
class LSTMClassifier(nn.Module):
    def __init__(self, num_classes, num_embeddings, embedding_dim, hidden_size, num_layers):
        super(LSTMClassifier, self).__init__()
        self.num_classes = num_classes
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers  # Make sure to define this attribute

        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])  # Take the last output for prediction
        return out

# Initialize the model
model = LSTMClassifier(num_classes, num_unique_strings, embedding_dim, hidden_size, num_layers)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 3  # Number of epochs

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Streaming output truncated to the last 5000 lines.
Epoch [1/3], Step [80/16698], Loss: 0.0002
Epoch [1/3], Step [90/16698], Loss: 0.0002
Epoch [1/3], Step [100/16698], Loss: 0.0002
Epoch [1/3], Step [110/16698], Loss: 0.0002
Epoch [1/3], Step [120/16698], Loss: 0.0002
Epoch [1/3], Step [130/16698], Loss: 0.0001
Epoch [1/3], Step [140/16698], Loss: 0.0001
Epoch [1/3], Step [150/16698], Loss: 0.0001
Epoch [1/3], Step [160/16698], Loss: 0.0001
Epoch [1/3], Step [170/16698], Loss: 0.0001
Epoch [1/3], Step [180/16698], Loss: 0.0001
Epoch [1/3], Step [190/16698], Loss: 0.0001
Epoch [1/3], Step [200/16698], Loss: 0.0001
Epoch [1/3], Step [210/16698], Loss: 0.0001
Epoch [1/3], Step [220/16698], Loss: 0.0001
Epoch [1/3], Step [230/16698], Loss: 0.0001
Epoch [1/3], Step [240/16698], Loss: 0.0001
Epoch [1/3], Step [250/16698], Loss: 0.0001
Epoch [1/3], Step [260/16698], Loss: 0.0001
Epoch [1/3], Step [270/16698], Loss: 0.0001
Epoch [1/3], Step [280/16698], Loss: 0.0001
Epoch [1/3], Step [290/1669

In [30]:
# Test loop
model.eval()  # Set the model to evaluation mode
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        test_loss += loss.item() * data.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Calculate and print test loss and accuracy
test_loss /= len(test_loader.dataset)
test_accuracy = 100 * correct / total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0000, Test Accuracy: 100.00%
